<span>
<img src="img/dynetx.png" width="220px" align="right"/>
</span>
<span>
<b>Author:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti</a><br/>
<b>Python version:</b>  >=3.6<br/>
<b>DyNetX version:</b>  0.0.1<br/>
<b>Last update:</b> 15/02/2021
</span>

<a id='top'></a>
# *Chapter 9: Network Dynamics*

In this notebook are introduced basilar snippet to cover dynamic networ modeling and analysis.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started.

<a id="dynetx"></a>
## DyNetX: a library for dynamic network modeling

So far we assumed that a *static* network topology. In real world scenario it is likely to observe nodes (as well as edges) that appear and desapear as time goes by, deeply affecting network structure and connectivity.

Indeed, topological transformations have huge implications on how diffusive phenomena unfold.

[``DyNetx``](http://dynetx.readthedocs.io/en/latest/) model time-evolving graphs. In the following we briefly introduce some [``DyNetx``](http://dynetx.readthedocs.io/en/latest/) primitives that allows to build and analyse dynamic networks.

A dynamic network is a topology having timestamps attached to edges (and/or nodes). As an example:

<img src="img/rete.png" width="50%" align="center"/>

[``DyNetx``](http://dynetx.readthedocs.io/en/latest/) is a Python software package that extends [``networkx``](https://networkx.github.io) with dynamic network models and algorithms.

We developed [``DyNetx``](http://dynetx.readthedocs.io/en/latest/) as a support library for ``NDlib``. It provides a generic implementation of dynamic network topology that can be used to model directed/undirected
- [Snapshot Graphs](#snapshots)
- [Interaction Networks](#interactions)

    pip install dynetx

<a id="snapshots"></a>
#### Snapshot Graphs ([to top](#top))

Often, network history is partitioned into a series of snap- shots, each one of them corresponding either to the state of the network at a time $t$ or to the aggregation of observed interactions during a period. Formally,

> A ``Snapshot Graph`` $G_t$ is defined by a temporally ordered set $⟨G_1, G_2\dots G_t⟩$ of static graphs where each snapshot $G_i = (V_i, E_i)$ is univocally identified by the sets of nodes $V_i$ and edges $E_i$.

Network snapshots can be effectively used, for instance, to model a phenomenon that generates network perturbations (almost) at regular intervals. In this scenario, context-dependent temporal windows are used to partition the network history into consecutive snapshots: time-bounded observations describing a precise, static, discretization of the network life.

Considering our dynamic network example we can identify the following snapshot graphs:

<img src="img/ex1.png" width="35%" align="left"/><img src="img/ex2.png" width="25%" align="left"/><img src="img/ex3.png" width="35%" align="left"/>

[``DyNetx``](http://dynetx.readthedocs.io/en/latest/) allows to (among the other things):
- List the snapshots of the loaded graph

<a id="interactions"></a>
#### Interaction networks ([to top](#top))

An ``Interaction network`` models a dynamic structure in which both nodes and edges may appear and disappear as time goes by. Usually, ``Intercation network`` are used in absence of a clear aggregation time scale, or when make sense to analyse a dynamic networok as a continuos stream of edges. Formally,

> An ``interaction network`` is a graph $G = (V, E, T)$ where: $V$ is a set of triplets of the form $(v, t_s, t_e)$, with $v$ a vertex of the graph and $t_s$, $t_e \in T$ are respectively the birth and death timestamps of the corresponding vertex (with $t_s \leq t_e$); $E$ is a set of quadruplets $(u, v, t_s, t_e)$, with $u, v \in V$ are vertices of the graph and $t_s,t_e \in T$ are respectively the birth and death timestamps of the corresponding edge (with $t_s \leq t_e$).

Considering our dynamic network example we can identify the following interaction stream:

<img src="img/ex4.png"  />

[``DyNetx``](http://dynetx.readthedocs.io/en/latest/) allows to to obtain the edge stream of a given dynamic graph.

In [1]:
!pip install networkx==2.8.8
!pip install dynetx

  Using cached networkx-2.8.8-py3-none-any.whl.metadata (5.1 kB)
Using cached networkx-2.8.8-py3-none-any.whl (2.0 MB)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
angelcommunity 2.0.0 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
cdlib 0.4.0 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.


In [ ]:
import dynetx as dn
import networkx as nx
import random

def read_net(filename):
    g = nx.Graph()
    with open(filename) as f:
        f.readline()
        for l in f:
            l = l.split(",")
            g.add_edge(l[0], l[1])
    return g

g = dn.DynGraph() # empty dynamic graph

In [ ]:
!unzip asioaf.zip

## Dynamic network creation

A dynamic network can be built by adding edges with specific appearence time (and eventually, vanishing time).

In our example, 10 ER graphs are generated and used to represent different topological evolutions of a same dynamic system.

In [ ]:
for t in range(1, 9):
    er = read_net(f'asioaf/got-s{t}-edges.csv')#nx.erdos_renyi_graph(random.randint(100, 400), 0.05)
    g.add_interactions_from(er.edges, t=t)

We can get the list of snapshot ids with

In [ ]:
g.temporal_snapshots_ids()

[1, 2, 3, 4, 5, 6, 7, 8]

Moreover, we can access each snapshot by its id

In [ ]:
g1 = g.time_slice(1)

In [ ]:
type(g1), g1.number_of_nodes(), g1.number_of_edges()

(dynetx.classes.dyngraph.DynGraph, 126, 549)

Following the same rationale it is possible to obtain timeslices covering any time window

In [ ]:
g0_3 = g.time_slice(0, 3)

In [ ]:
type(g0_3), g0_3.number_of_nodes(), g0_3.number_of_edges(), g0_3.interactions_per_snapshots()

(dynetx.classes.dyngraph.DynGraph, 237, 1182, {1: 78.5, 2: 79.0})

If the slice cover a single snapshot it can be analyzed transforming it in a ``networkx`` object, otherwise ``dynetx`` methods need to be applied

In [ ]:
g1_flat = nx.Graph(g1.edges())

In [ ]:
type(g1_flat), g1_flat.number_of_nodes(), g1_flat.number_of_edges()

(networkx.classes.graph.Graph, 126, 549)

### Dynamic network measures

#### Inter event time (Global)

Distribution of inter event time (e.g., how much time before a new interaction appears in the graph)

In [ ]:
r = g.inter_event_time_distribution()
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 3307, 1: 8}


#### Inter event time (Node)

Distribution of inter event time (e.g., how much time before a new interaction involving a specific node appears in the graph)

In [ ]:
r = g.inter_event_time_distribution("ARYA")
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 137, 1: 8}


#### Inter event time (Edge)

Distribution of inter event time (e.g., how much time before a new interaction among two nodes, u and v, appears in the graph)

In [ ]:
u = 'JON'
v = 'ARYA'

In [ ]:
r = g.inter_event_time_distribution(u, v)
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{6: 1, 1: 1}


### Degree
Degrees can be queried time-wise

In [ ]:
g.degree(t=2)['ARYA'] # degree of node 0 at time t=2

27

### Coverage

The ratio of existing nodes w.r.t. the possible ones.

In [ ]:
g.coverage()

0.2977216748768473

#### Node contribution

Node u coverage of the temporal graph.

In [ ]:
g.node_contribution("BERIC")

0.625

#### Edge contribution

Edge (u, v) coverage of the temporal graph.

In [ ]:
g.edge_contribution(u, v)

0.375

#### Node pair uniformity

Overlap between the presence times of u and v.

In [ ]:
g.node_pair_uniformity(u, v)

1.0

### Density
Temporal network density: fraction of possible interactions that do exist in the temporal network.

In [ ]:
g.density()

0.06686633244351846

#### Node Density
Intersection among the temporal presence of the edge (u, v) and the joint temporal presences of u and v.

In [ ]:
g.node_density(u)

0.2295760082730093

#### Pair Density

Intersection among the temporal presence of the edge (u, v) and the joint temporal presences of u and v.

In [ ]:
g.pair_density(u, v)

0.375

#### Snapshot Density

Density of a temporal network at time t.

In [ ]:
for t in g.temporal_snapshots_ids():
    print(f"{t}\t{g.snapshot_density(t)}")

1	0.06971428571428571
2	0.05886627906976744
3	0.06608969315499606
4	0.04535563715490276
5	0.05640222190571144
6	0.05404055538907202
7	0.1271604938271605
8	0.20473898556090336


### Path analysis

Computes the time respecting paths among u and v within [start, stop]

In [ ]:
import dynetx.algorithms as al
paths = al.time_respecting_paths(g, "GENDRY", "GREY_WORM", start=1, end=5)

In [ ]:
p = paths[("GENDRY", "GREY_WORM")][0] # example of identified paths. Each list element is a tuple of the form (from, to, time)
p

(('GENDRY', 'NED', 1),
 ('NED', 'ROBERT', 2),
 ('ROBERT', 'BARRISTAN', 3),
 ('BARRISTAN', 'GREY_WORM', 4))

Moreover, it is possible to compute length and duration of a given path

In [ ]:
al.path_duration(p), al.path_length(p)

(3, 4)

Among all paths it is possible to identify the most interestin ones using

In [ ]:
annotated = al.annotate_paths(paths[("GENDRY", "GREY_WORM")])

In [ ]:
annotated['shortest']

[(('GENDRY', 'VARYS', 1), ('VARYS', 'GREY_WORM', 6))]

In [ ]:
annotated['fastest']

[(('GENDRY', 'TYWIN', 3), ('TYWIN', 'TYRION', 4), ('TYRION', 'GREY_WORM', 5))]

In [ ]:
annotated['shortest_fastest']

[[('GENDRY', 'TYWIN', 3), ('TYWIN', 'TYRION', 4), ('TYRION', 'GREY_WORM', 5)]]

In [ ]:
annotated['fastest_shortest']

[[('GENDRY', 'VARYS', 1), ('VARYS', 'GREY_WORM', 6)]]

In [ ]:
annotated['foremost']

[(('GENDRY', 'NED', 1),
  ('NED', 'ROBERT', 2),
  ('ROBERT', 'BARRISTAN', 3),
  ('BARRISTAN', 'GREY_WORM', 4)),
 (('GENDRY', 'NED', 1),
  ('NED', 'ROBERT', 2),
  ('ROBERT', 'DAENERYS', 3),
  ('DAENERYS', 'GREY_WORM', 4)),
 (('GENDRY', 'NED', 1),
  ('NED', 'ROBERT', 2),
  ('ROBERT', 'JORAH', 3),
  ('JORAH', 'GREY_WORM', 4))]